In [1]:
import ollama
import pandas as pd
from tqdm import tqdm
from pydantic import BaseModel
from sklearn.metrics import confusion_matrix
import numpy as np

def ollamaResponse(content):
    response = ollama.chat(model='llama3', messages=[
    {
        'role': 'user',
        'content': content,
    },
    ])
    return response['message']['content']

class Response(BaseModel):
    response: int

ModuleNotFoundError: No module named 'ollama'

In [ ]:
path = '/Users/Alex/Desktop/2.FutureTech/OSFM/uniform_sample/results/9-13-and-8-28-with-confidence.csv'
df = pd.read_csv(path)
label = 'alex'

df_extends = df[(df['classification'] == 'extends')]


In [ ]:
prompt = "The following sentences as from an academic paper, and cites a foundation model indicated by these <cite> citation brackets </cite>. An automated classifier determined that teh sentences mention fine-tune or training the cited model. Now suppose that you are the author of these sentences. Base on the following sentences, respond in the JSON format {{\"response\": bool}} whether you were the one who fine-tuned the model (true) or whether someone else did (false). Give one response for the whole set of sentences, without explanation. The sentence is as follows: {sentence}"

def is_misattribution(sentence):
    return ollamaResponse(prompt.format(sentence = sentence))

misattributions = []
iters = 1
for sentence in tqdm(df_extends['multisentence'], total = len(df_extends)):
    responses = [is_misattribution(sentence).lower() for i in range(iters)]
    models = map(Response.model_validate_json, responses)
    total = np.array([model.response for model in models]).sum()
    misattributions.append(total)
    
df_extends['misattributions'] = misattributions


In [ ]:
threshold = 2
df_extends['corrected_classification'] = [c if misatt < threshold else 'context' for c, misatt in zip(df_extends['classification'], df_extends['misattributions'])]
mask = (pd.to_numeric(df_extends['alex_confidence'], errors = 'coerce') >= 5) | df_extends['alex_confidence'].isna()
df_filtered = df_extends[mask]
cm = confusion_matrix(df_filtered[label], df_filtered['corrected_classification'])
cm

In [ ]:
df_extends[df_extends['classification'] == 'context']